Notebook voor het genereren van data voor het trainen van Tesseract-OCR. De package 'trdg' is hiervoor zelf aangepast om data te genereren in kenteken format. 

In [12]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 119
%load_ext autoreload
%autoreload 2

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import matplotlib.pyplot as plt
import os
import re
import random
import string
import PIL
from random import randrange

In [14]:
path = r"../DataSet/TesseractTraining/"

In [15]:
def create_strings_randomly(count=10, let=True, num=True, lang='eng'):
    """
    Create all strings by randomly sampling from a pool of characters.

    :param count: amount of pictures to generate
    :param let: wheter to use letters or not
    :param num: whether to use numbers or not
    :param lang: language

    :type count: int
    :type let: bool
    :type num: bool
    :type lang: string

    :return strings: generated strings
    :rtype strings: List of strings
    """

    pool = ""
    if let:
        if lang == "cn":
            pool += "".join(
                [chr(i) for i in range(19968, 40908)]
            )  # Unicode range of CHK characters
        else:
            pool += string.ascii_letters
    if num:
        pool += "0123456789"

    strings = []
    for _ in range(0, count):
        current_string = "-XXX-"
        front = randrange(1, 4)*"X"
        back = randrange(1, 4)*"X"
        current_string = front+current_string+back
        current_string = list(current_string)
        i = -1
        for letter in current_string:
            i += 1
            if letter == "-":
                continue
            current_string[i] = random.choice(pool).upper()
        current_string = "".join(current_string)
        strings.append(current_string)
    return strings

In [16]:
def create_new_img(string, skewing_angle=0, blur=0):
    """
    Creates a new image

    :param string: string that will be on the picture
    :param skewing_angle: maximum amount of skewing that can be added
    :param blur: maximum amount of blur that can be added

    :type string: string
    :type skewing_angle: int
    :type blur: int
    """
    # Load image
    img = PIL.Image.open(path+r"plate_template/plate1.jpg")
    draw = PIL.ImageDraw.Draw(img)

    # Set font size depending on length of string
    if len(string) >= 10:
        font = PIL.ImageFont.truetype("GIL_____.TTF", 90)
    else:
        font = PIL.ImageFont.truetype("GIL_____.TTF", 100)
    width, height = img.size

    # Draw the text
    draw.text(((width/2)+20, (height/2)-10), string, (0, 0, 0), font=font, anchor="mm")

    # Random angle
    random_angle = random.randint(0 - skewing_angle, skewing_angle)
    img = img.rotate(random_angle)

    # Random blurring
    gaussian_filter = PIL.ImageFilter.GaussianBlur(random.randint(0, blur))
    img = img.filter(gaussian_filter)

    img.save(path+r"plate_model_6-ground-truth/"+string+".png")

In [17]:
def plate_generator(count=10, skewing_angle=0, blur=0):
    """
    :param count: amount of pictures to generate
    :param skewing_angle: maximum amount of skewing that can be added
    :param blur: maximum amount of blur that can be added

    :type count: int
    :type skewing_angle: int
    :type blur: int
    """
    strings = create_strings_randomly(count)
    print(len(strings))

    counter = 0
    for string in strings:
        counter += 1
        if counter % 500 == 0:
            print("Counter is at", counter)

        create_new_img(string, skewing_angle, blur)
        text = re.sub(r'[^\w]', ' ', string)
        with open(path+r"plate_model_6-ground-truth/"+string+".gt.txt", "w") as text_file:
            text_file.write(text)

In [18]:
plate_generator(count=5, skewing_angle=5, blur=5)

5


# Bron
https://github.com/Belval/TextRecognitionDataGenerator